# Estimation of Navier-Stokes equation
## Done by: Andreea-Ioana Florea

In this project I aim at developing a PINN for estimation of the 2D Navier-Stokes equation.

# Imports and dependencies and whatnot

In [48]:
!pip install deepxde

In [60]:
import tensorflow as tf
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

# Parameters

In [61]:
rho = 1  # Density
mu = 0.005   # Viscosity
u_in = 1
D = 1
L = 2
t_0 = 0
t_1 = 2
num_domain=10000
num_boundary=5000
num_initial=1000

In [62]:
channel = dde.geometry.Rectangle(xmin=[-L / 2, -D / 2], xmax=[L / 2, D / 2])
cylinder = dde.geometry.Disk([0, 0], 0.1)

geom_space = channel - cylinder

geom_time = dde.geometry.TimeDomain(t_0, t_1)
geom = dde.geometry.GeometryXTime(geom_space, geom_time)

In [63]:
def pde(x, y):
    u = y[:, 0:1]
    v = y[:, 1:2]
    p = y[:, 2:3]

    u_x = dde.grad.jacobian(y, x, i=0, j=0)
    u_y = dde.grad.jacobian(y, x, i=0, j=1)
    u_t = dde.grad.jacobian(y, x, i=0, j=2)

    v_x = dde.grad.jacobian(y, x, i=1, j=0)
    v_y = dde.grad.jacobian(y, x, i=1, j=1)
    v_t = dde.grad.jacobian(y, x, i=1, j=2)

    p_x = dde.grad.jacobian(y, x, i=2, j=0)
    p_y = dde.grad.jacobian(y, x, i=2, j=1)

    u_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    u_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)

    v_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    v_yy = dde.grad.hessian(y, x, component=1, i=1, j=1)

    continuity = u_x + v_y

    mom_u = u_t + u * u_x + v * u_y + p_x / rho - mu * (u_xx + u_yy)
    mom_v = v_t + u * v_x + v * v_y + p_y / rho - mu * (v_xx + v_yy)

    return [continuity, mom_u, mom_v]

In [64]:
def inlet_boundary(x, on_boundary):
    return on_boundary and np.isclose(x[0], -L / 2)

def outlet_boundary(x, on_boundary):
    return on_boundary and np.isclose(x[0], L / 2)

def top_bottom_boundary(x, on_boundary):
    return on_boundary and (np.isclose(x[1], D / 2) or np.isclose(x[1], -D / 2))

def cylinder_boundary(x, on_boundary):
    return on_boundary and (x[0] ** 2 + x[1] ** 2 <= (0.1 + 1e-6) ** 2)

def inlet_u(x):
    y = x[:, 1:2]
    return 4 * u_in * (y + D / 2) * (D / 2 - y) / D ** 2

In [65]:
bc_inlet_u = dde.DirichletBC(geom, inlet_u, inlet_boundary, component=0)
bc_inlet_v = dde.DirichletBC(geom, lambda x: 0.0, inlet_boundary, component=1)

bc_wall_u = dde.DirichletBC(geom, lambda x: 0.0, top_bottom_boundary, component=0)
bc_wall_v = dde.DirichletBC(geom, lambda x: 0.0, top_bottom_boundary, component=1)

bc_cyl_u = dde.DirichletBC(geom, lambda x: 0.0, cylinder_boundary, component=0)
bc_cyl_v = dde.DirichletBC(geom, lambda x: 0.0, cylinder_boundary, component=1)

bc_outlet_p = dde.DirichletBC(geom, lambda x: 0.0, outlet_boundary, component=2)

ic_u = dde.IC(geom, lambda x: 0.0, lambda x, on_initial: on_initial, component=0)
ic_v = dde.IC(geom, lambda x: 0.0, lambda x, on_initial: on_initial, component=1)
ic_p = dde.IC(geom, lambda x: 0.0, lambda x, on_initial: on_initial, component=2)

bcs = [bc_inlet_u, bc_inlet_v, bc_wall_u, bc_wall_v, bc_cyl_u, bc_cyl_v, bc_outlet_p, ic_u, ic_v, ic_p]

In [66]:
data = dde.data.TimePDE(
    geom,
    pde,
    bcs,
    num_domain=num_domain,
    num_boundary=num_boundary,
    num_initial=num_initial
)

# Neural Network

In [67]:
net = dde.maps.FNN([3] + [64] * 3 + [3], "tanh", "Glorot uniform")
model = dde.Model(data, net)

In [75]:
model.compile("adam", lr=1e-3)
losshistory, train_state = model.train(epochs=5000)

Compiling model...
'compile' took 0.005842 s

Training model...

Step      Train loss                                                                                                                            Test loss                                                                                                                             Test metric
30000     [6.88e-03, 4.08e-03, 1.03e-03, 5.66e-03, 2.94e-04, 1.16e-03, 1.27e-02, 1.27e-03, 2.43e-04, 1.06e-05, 2.26e-03, 3.39e-04, 1.46e-04]    [6.88e-03, 4.08e-03, 1.03e-03, 5.66e-03, 2.94e-04, 1.16e-03, 1.27e-02, 1.27e-03, 2.43e-04, 1.06e-05, 2.26e-03, 3.39e-04, 1.46e-04]    []  
31000     [6.90e-03, 4.13e-03, 1.04e-03, 5.43e-03, 2.91e-04, 1.20e-03, 1.27e-02, 1.38e-03, 2.43e-04, 1.08e-05, 2.25e-03, 3.36e-04, 1.42e-04]    [6.90e-03, 4.13e-03, 1.04e-03, 5.43e-03, 2.91e-04, 1.20e-03, 1.27e-02, 1.38e-03, 2.43e-04, 1.08e-05, 2.25e-03, 3.36e-04, 1.42e-04]    []  
32000     [6.82e-03, 5.70e-03, 1.69e-03, 4.58e-03, 3.54e-04, 1.70e-03, 1.25e-02

In [78]:
model.compile("L-BFGS-B")
losshistory, train_state = model.train()

Compiling model...
'compile' took 0.001550 s

Training model...

Step      Train loss                                                                                                                            Test loss                                                                                                                             Test metric
35000     [8.03e-03, 4.91e-03, 1.82e-03, 4.67e-03, 3.37e-04, 1.19e-03, 1.21e-02, 1.34e-03, 3.54e-04, 6.23e-05, 2.15e-03, 3.19e-04, 2.21e-04]    [8.03e-03, 4.91e-03, 1.82e-03, 4.67e-03, 3.37e-04, 1.19e-03, 1.21e-02, 1.34e-03, 3.54e-04, 6.23e-05, 2.15e-03, 3.19e-04, 2.21e-04]    []  


KeyboardInterrupt: 

In [86]:
model.save("navier_stokes_pinn")

'navier_stokes_pinn-35000.weights.h5'

# Visualization

In [ ]:
import matplotlib.animation as animation
from matplotlib. patches import Circle

In [ ]:
nx, ny = 200, 100
x = np.linspace(-L/2, L/2, nx)
y = np.linspace(-D/2, D/2, ny)
X, Y = np.meshgrid(x, y)
XY = np.vstack([X.ravel(), Y.ravel()]).T

n_frames = 900
t_vals = np.linspace(t_0, t_1, n_frames)

R = 0.1
cyl_mask = (X**2 + Y**2) <= R**2

def get_flow_data_at_t(t):
    """Get velocity components and speed at time t"""
    t_col = np.full((XY.shape[0], 1), t)
    XYT = np. hstack([XY, t_col])
    pred = model.predict(XYT)
    u = pred[:, 0]. reshape(ny, nx)
    v = pred[:, 1]. reshape(ny, nx)
    speed = np.sqrt(u**2 + v**2)
    # Mask the cylinder region
    u = np.where(cyl_mask, np.nan, u)
    v = np.where(cyl_mask, np.nan, v)
    speed = np.where(cyl_mask, np. nan, speed)
    return u, v, speed

# Precompute all frames
flow_data = [get_flow_data_at_t(t) for t in t_vals]
speeds = [data[2] for data in flow_data]
vmin = min(np.nanmin(s) for s in speeds)
vmax = max(np.nanmax(s) for s in speeds)

fig, ax = plt.subplots(figsize=(8, 4))
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim(-L/2, L/2)
ax.set_ylim(-D/2, D/2)
ax.set_aspect('equal')
title = ax.set_title(f"Speed magnitude with streamlines, t={t_vals[0]:.2f}")

# Add cylinder patch
cyl = Circle((0, 0), R, edgecolor="black", facecolor="white", zorder=10)
ax.add_patch(cyl)

# Initial plot
u0, v0, speed0 = flow_data[0]
cax = ax.contourf(X, Y, speed0, levels=50, cmap="jet", vmin=vmin, vmax=vmax)
cb = fig.colorbar(cax, ax=ax)
cb.set_label("Speed magnitude")

# Initial streamlines
stream = ax.streamplot(X, Y, u0, v0, color='white', linewidth=0.6,
                        density=1.5, arrowsize=0.8, arrowstyle='->')

def update(frame):
    # Clear previous contours and streamlines
    for coll in list(ax.collections):
        coll.remove()
    # Clear streamline arrows (they are stored in patches)
    for patch in list(ax.patches):
        if not isinstance(patch, Circle):
            patch.remove()

    u, v, speed = flow_data[frame]

    # Redraw contours
    ax.contourf(X, Y, speed, levels=50, cmap="jet", vmin=vmin, vmax=vmax)

    # Redraw streamlines
    # Create specific starting points along the inlet
    n_lines = 30  # Number of streamlines
    start_y = np.linspace(-D/2 + 0.05, D/2 - 0.05, n_lines)
    start_x = np.full_like(start_y, -L/2 + 0.01)
    start_points = np.column_stack([start_x, start_y])

    ax.streamplot(X, Y, u, v, color='white', linewidth=0.8,
              start_points=start_points, arrowsize=0.8, arrowstyle='->')

    # Re-add cylinder (to ensure it's on top)
    cyl = Circle((0, 0), R, edgecolor="black", facecolor="white", zorder=10)
    ax.add_patch(cyl)

    title.set_text(f"Speed magnitude with streamlines, t={t_vals[frame]:.2f}")
    return None

ani = animation. FuncAnimation(fig, update, frames=n_frames, blit=False)

ani.save("animation.mp4", writer="ffmpeg", dpi=300, fps=60)
plt.close(fig)